#### Let us build the propht model for the selected city_medicines 

In [2]:
import numpy as np
import pandas as pd
import json
import math
import os
from fbprophet import Prophet
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error

In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
path=os.getcwd()
path

'C:\\Users\\jayam\\Downloads\\7879phd'

In [4]:
test=pd.read_csv(path +"\\test_final.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])
train=pd.read_csv(path +"\\train_final.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])

In [5]:
train.head()

,year,month,day,city,medicine,date,sales,discounted,year_month,city_medicine,week,footfall_perday_permed,old_new
0,2015,1,2,1,1,2015-01-02,24.0,0,2015_01,1_1,1,0.385804,old
1,2015,1,2,1,2,2015-01-02,144.0,0,2015_01,1_2,1,2.314824,old
2,2015,1,2,1,3,2015-01-02,84.0,0,2015_01,1_3,1,1.350314,old
3,2015,1,2,1,4,2015-01-02,24.0,0,2015_01,1_4,1,0.385804,old
4,2015,1,2,1,5,2015-01-02,8.0,0,2015_01,1_5,1,0.128601,old


In [6]:
df=pd.DataFrame(test["city_medicine"].value_counts(normalize=True))
df.rename(columns={"city_medicine":"percent"},inplace=True)
df["cumsum"]=df["percent"].cumsum()*100
df["valuecount"]=test["city_medicine"].value_counts().values

In [10]:
df[df["valuecount"]==31].tail()

,percent,cumsum,valuecount
6_1937,0.00004,42.496332,31
5_371,0.00004,42.500339,31
6_1102,0.00004,42.504347,31
4_1315,0.00004,42.508355,31
4_2009,0.00004,42.512362,31


In [11]:
df[df["valuecount"]==31].shape

(10608, 3)

#### Let us build the propht model for all these 10608 medicines as they contribute to 42 percent of the rows in test dataset 

In [12]:
top_42=list(df[df["valuecount"]==31].index)

In [13]:
top_42_df=df.iloc[:10608,]

In [15]:
top_42_df.tail()

,percent,cumsum,valuecount
6_1937,0.00004,42.496332,31
5_371,0.00004,42.500339,31
6_1102,0.00004,42.504347,31
4_1315,0.00004,42.508355,31
4_2009,0.00004,42.512362,31


- In general propht model requires 1 year data gives good modelling results 
- so let us consider the data from 2017

In [16]:
trn=train[train["year"]>=2017]

In [17]:
np.unique(top_42_df.index.isin(list(trn["city_medicine"].values)),return_counts=True)

(array([False,  True]), array([    3, 10605], dtype=int64))

In [18]:
np.unique(top_42_df.index.isin(list(train["city_medicine"].values)),return_counts=True)

(array([False,  True]), array([    3, 10605], dtype=int64))

- so out of 10608 which are contributing to 42% of rows in test data there are 3 medicines which are not available in both the from 2017 data and entire  train dataset

In [26]:
cmn_meds_df=top_42_df[top_42_df.index.isin(list(trn["city_medicine"].values))]

In [27]:
cmn_meds_df.shape

(10605, 3)

In [28]:
cmn_trn_df=trn[trn["city_medicine"].isin(cmn_meds_df.index)]

In [29]:
cmn_trn_df.head()

,year,month,day,city,medicine,date,sales,discounted,year_month,city_medicine,week,footfall_perday_permed,old_new
10882319,2017,1,2,1,2,2017-01-02,72.000,0,2017_01,1_2,1,0.878203,old
10882320,2017,1,2,1,3,2017-01-02,128.000,0,2017_01,1_3,1,1.561250,old
10882323,2017,1,2,1,8,2017-01-02,64.000,0,2017_01,1_8,1,0.780625,old
10882327,2017,1,2,1,13,2017-01-02,52.000,0,2017_01,1_13,1,0.634258,old
10882328,2017,1,2,1,14,2017-01-02,47.692,1,2017_01,1_14,1,0.581712,old


In [31]:
sub_set=cmn_trn_df[["date","sales","city_medicine"]]

In [32]:
lst=list(cmn_trn_df["city_medicine"].value_counts().index)

In [34]:
sub_set.reset_index(inplace=True,drop=True)

In [36]:
sub_set.head()

,date,sales,city_medicine
0,2017-01-02,72.000,1_2
1,2017-01-02,128.000,1_3
2,2017-01-02,64.000,1_8
3,2017-01-02,52.000,1_13
4,2017-01-02,47.692,1_14


In [37]:
def add_dates(lst):# adding missing dates to the data
    final=pd.DataFrame()
    count=0
    for med in lst:
        df=sub_set[sub_set["city_medicine"]==med]# subset the data frame
        df["date"]=pd.to_datetime(df["date"])# converting to date time
        start_date1=pd.to_datetime("2017-01-02")# selecting the starting date
        end_date = pd.to_datetime("2018-06-30")# selecting the end date
        dates=pd.DataFrame()# creating dates data frame
        dates["date"]=pd.date_range(start=start_date1,end=end_date,freq="d")# generating date range
        dates=dates.merge(df,on=["date"],how="left")# merging dates dataframe with df 
        dates["city_medicine"]=med
        dates["sales"]=dates["sales"].fillna(0)# fill na values of sales with zero
        #print(dates.head())
        final=pd.concat([final,dates])
    return final

In [38]:
new_trn_df=add_dates(lst)

In [39]:
new_trn_df.to_csv("new_trn_df.csv",index=False)

In [40]:
def propht_model(lst):# creating a fuction for running the model on each city_medicine
    fin=pd.DataFrame()
    for med in lst:
        df=new_trn_df[new_trn_df["city_medicine"]==med]# subset the data frame
        df.columns = ['ds', 'y',"city_medicine"]# rename columns as it is a requiremnt for propht model
        #print(med)
        model = Prophet(interval_width = 0.95)# we can also get confidence interval for each of our predictions (95% confidence interval)
        model.fit(df)
        future= model.make_future_dataframe(periods=31)# we need predictions for next 31 days
        forecast = model.predict(future[-31:])# we are storing only last 31 days of predictions 
        forecast["city_medicine"]=med
        fin=pd.concat([fin,forecast[["ds","yhat","city_medicine"]]])
    return fin

In [41]:
pred=propht_model(lst)

In [58]:
pred.to_csv("pred_to_testmerg.csv",index=False)

In [146]:
prds.columns

Index(['yhat', 'ds', 'city_medicine'], dtype='object')

#### Let us merge the predictions with our best model, this is is the best model which has scored 50.52 in SCT and also combining with new city medicine average predictions

Conclusion : we can fruther improve the model by strategically selecting the the city medicine combinations with p-values for which we can run propht model